In [1]:
import os
import caesar
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import modules as anal
import modules.anal_func as anal_func

from astropy.cosmology import LambdaCDM
import astropy.units as u
from astropy.io import fits
from scipy import interpolate

from modules.powdermod.makesed import MakeSED

cosmo = LambdaCDM(H0 = 68, Om0= 0.3, Ode0= 0.7, Ob0=0.048)
plt.rcParams.update({'font.size': 28})

sb = anal.Simba('cis', size='50_full')

In [ ]:
# select the galaxies for the SED

In [ ]:
cs = sb.get_caesar(104)
id = np.asarray([i.GroupID for i in cs.galaxies])
d = np.asarray([i.masses['dust'] for i in cs.galaxies])
g = np.asarray([i.masses['H2'] for i in cs.galaxies])
m = np.asarray([i.masses['stellar'] for i in cs.galaxies])
sfr = np.asarray([i.sfr for i in cs.galaxies])

mk = np.all([np.log10(sfr/m) < -11, m > 1e10], axis=0)
plt.scatter(np.log10(d/m)[mk], np.log10(g/m)[mk])
plt.savefig('scatter_dust_gas.png')

print(id[np.where(np.log10(d/m)[mk]==np.log10(d/m)[mk].max())])
print(id[np.where(np.log10(d/m)[mk]<-3)])
print(np.log10(d/m)[151])

In [2]:
# initialize SED making class
hydro_dir_base = '/mnt/home/glorenzon/simbanator/output/hdf5/filtered_part_files/'
preselect = '/mnt/home/glorenzon/simbanator/output/txt/target_selection_for_SED/test_selection.txt'
selection_file = 'selection_compare_jwst_21'
makesed = MakeSED(sb, nnodes=1, model_run_name='m50', hydro_dir_base=hydro_dir_base, selection_file=selection_file, preselect=preselect)

In [3]:
makesed.selection_gals([100], [1]) # create the hdf5 selection
makesed.create_master('cluster', 'region', 50) # create the parameters masters

yt : [INFO     ] 2024-10-26 15:00:24,803 Opening /mnt/home/glorenzon/SIMBA_catalogs/simba50_snaps/full//Groups/m50n512_100.hdf5
yt : [INFO     ] 2024-10-26 15:00:24,876 Found 66852 halos
yt : [INFO     ] 2024-10-26 15:00:24,921 Found 4730 galaxies


Inserted aggregated data for: snap100
===================== snap100/code_coods [[11216.85367188  8394.57078125 19224.71164063]]
Processing galaxy:  1  in snap  100
Initializing position:  [11216.85367188  8394.57078125 19224.71164063]
writing slurm submission master script file
/mnt/home/glorenzon/simbanator/output/hdf5/powderday_sed_out/snap_100/master.snap100.job


In [ ]:
fig, ax = makesed.plotsed('090', 0, ret=True)

In [ ]:
ax.set_ylim(1e-10, 1e11)
fig.savefig('again.png')

In [ ]:
cs = sb.get_caesar(104)

In [ ]:
[i.pos.in_units('code_length').value for i in cs.galaxies][0]

In [ ]:
coods_code = np.array([g.pos.in_units('code_length').value for g in cs.galaxies])

In [ ]:
path_to_im = '/mnt/home/glorenzon/simbanator/output/hdf5/powderday_sed_out/snap_090/gal_0/convolved.090.hdf5'
import h5py
f = h5py.File(path_to_im, 'r')


In [ ]:
convolved_image = f['image_data'][1]
filter_name = f['filter_names'][1]
print(filter_name)

In [ ]:
f['g_SDSS.filter'][...]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as colors

fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111)

w = f['image_data'].attrs['width']
w_unit = f['image_data'].attrs['width_unit'].astype(str)
cax = ax.imshow(np.log(convolved_image), cmap=plt.cm.viridis,
                origin='lower', extent=[-w, w, -w, w])
ax.tick_params(axis='both', which='major', labelsize=10)
ax.set_xlabel('x ({})'.format(w_unit))
ax.set_ylabel('y ({})'.format(w_unit))
ax.set_xlim(-10, 10)
ax.set_ylim(-10, 10)
plt.colorbar(cax, label='log Luminosity (ergs/s)', format='%.0e')
plt.title("Convolved image: {}".format(filter_name))
plt.tight_layout()
plt.savefig('first_light_g.png')
plt.show()